In [1]:
import json
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow

In [2]:
work_dir = Path.home() / 'Programming/Python/machine-learning-exercises/higher-education-students-performance-evaluation'
data_file = work_dir / 'data/students-performance.csv'

In [3]:
attribute_names_json_file = work_dir / 'attribute_names.json'
with open(attribute_names_json_file, 'rt') as f_in:
    attribute_names_json = json.load(f_in)

In [4]:
labels_dict = {}
string_indexes = [str(id) for id in range(1, 33)]
for ind in string_indexes:
    label = attribute_names_json[ind]['name']
    labels_dict[ind] = label
labels_dict['0'] = 'STUDENT ID'

In [5]:
sp_df = pd.read_csv(data_file)
sp_df.rename(columns=labels_dict, inplace=True)

In [6]:
X = sp_df.drop(['STUDENT ID', 'GRADE'], axis=1)
y = sp_df['GRADE'].copy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=33)

In [8]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [9]:
cat_attribs = sp_df.columns[1:-1]
cat_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preprocessing = ColumnTransformer([("cat", cat_pipeline, cat_attribs)])

In [10]:
X_train_tr = preprocessing.fit_transform(X_train)
X_test_tr = preprocessing.transform(X_test)

In [11]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('higher-education-students-performance-evaluation');

## XGBoost Classifier

In [12]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'XGBoost')
        mlflow.log_params(params)
        clf = xgb.XGBClassifier(
            **params,
            eval_metric='auc',
            early_stopping_rounds=50,
            n_jobs=-1,
        )
        clf.fit(
            X_train_tr,
            y_train,
            eval_set=[(X_test_tr, y_test)],
        )
        y_pred = clf.predict_proba(X_test_tr)
        auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
        mlflow.log_metric("AUC", auc)

    return {'loss': auc, 'status': STATUS_OK}

In [13]:
xgboost_search_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 4, 200, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 4, 50, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'binary:logistic',
    'seed': 42
}

In [14]:
xgboost_results = fmin(
    fn=objective,
    space=xgboost_search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation_0-auc:0.60260                                                                                                                      
[1]	validation_0-auc:0.62415                                                                                                                      
[2]	validation_0-auc:0.63764                                                                                                                      
[3]	validation_0-auc:0.64801                                                                                                                      
[4]	validation_0-auc:0.64034                                                                                                                      
[5]	validation_0-auc:0.62230                                                                                                                      
[6]	validation_0-auc:0.61635                                                                                          

In [15]:
xgboost_best_params = {
    'learning_rate': 0.1822609570893024,
    'max_depth': 16,
    'min_child_weight': 0.4639113171017813,
    'n_estimators': 131,
    'objective': 'binary:logistic',
    'reg_alpha': 0.007860242176975434,
    'reg_lambda': 0.02768073078548693,
    'seed': 42,
}

In [16]:
with mlflow.start_run():
    mlflow.set_tag('model', 'XGBoost')
    mlflow.log_params(xgboost_best_params)
    xgboost_clf = xgb.XGBClassifier(
        **xgboost_best_params,
        eval_metric='auc',
        early_stopping_rounds=50,
        n_jobs=-1,
    )
    xgboost_clf.fit(
        X_train_tr,
        y_train,
        eval_set=[(X_test_tr, y_test)],
    )
    y_pred = xgboost_clf.predict_proba(X_test_tr)
    auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
    mlflow.log_metric("AUC", auc)

    with open('models/xgb_cls.bin', 'wb') as f_out:
        pickle.dump(xgboost_clf, f_out)

    mlflow.log_artifact('models/xgb_cls.bin', artifact_path='best_models')

    mlflow.xgboost.log_model(xgboost_clf, artifact_path='artifacts')

[0]	validation_0-auc:0.62236
[1]	validation_0-auc:0.64030
[2]	validation_0-auc:0.65390
[3]	validation_0-auc:0.68579
[4]	validation_0-auc:0.71081
[5]	validation_0-auc:0.72131
[6]	validation_0-auc:0.71703
[7]	validation_0-auc:0.72751
[8]	validation_0-auc:0.72040
[9]	validation_0-auc:0.70807
[10]	validation_0-auc:0.70215
[11]	validation_0-auc:0.69231
[12]	validation_0-auc:0.68800
[13]	validation_0-auc:0.68626
[14]	validation_0-auc:0.67731
[15]	validation_0-auc:0.67533
[16]	validation_0-auc:0.67299
[17]	validation_0-auc:0.66811
[18]	validation_0-auc:0.67082
[19]	validation_0-auc:0.67400
[20]	validation_0-auc:0.67182
[21]	validation_0-auc:0.67191
[22]	validation_0-auc:0.66840
[23]	validation_0-auc:0.66679
[24]	validation_0-auc:0.66564
[25]	validation_0-auc:0.66342
[26]	validation_0-auc:0.66268
[27]	validation_0-auc:0.65876
[28]	validation_0-auc:0.65653
[29]	validation_0-auc:0.65508
[30]	validation_0-auc:0.65281
[31]	validation_0-auc:0.65233
[32]	validation_0-auc:0.65164
[33]	validation_0-au

/opt/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:44:37] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1744329043786/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/07/14 15:44:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Logistic Regression

In [17]:
log_reg_params = {
    'C': scope.int(hp.quniform('C', -3, 3, 1)),
    'class_weight': 'balanced',
    'seed': 42
}

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'LogisticRegression')
        mlflow.log_params(params)
        log_reg = LogisticRegression()
        log_reg.fit(
            X_train_tr,
            y_train,
            )
        y_pred = log_reg.predict_proba(X_test_tr)
        auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
        mlflow.log_metric("AUC", auc)
        
        with open('models/log_reg.bin', 'wb') as f_out:
            pickle.dump(xgboost_clf, f_out)

        mlflow.log_artifact('models/log_reg.bin', artifact_path='best_models')

    return {'loss': auc, 'status': STATUS_OK}

In [19]:
log_reg_result = fmin(
    fn=objective,
    space=log_reg_params,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

100%|███████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 26.10trial/s, best loss: 0.5896990311276026]


## SVC

In [20]:
svc_params = {
    'C': scope.int(hp.quniform('C', -3, 3, 1)),
    'max_iter': scope.int(hp.quniform('max_iter', 1, 3, 1))
}

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'SVC')
        mlflow.log_params(params)
        svc_clf = SVC(probability=True)
        svc_clf.fit(
            X_train_tr,
            y_train,
            )
        y_pred = svc_clf.predict_proba(X_test_tr)
        auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
        mlflow.log_metric("AUC", auc)

        with open('models/svc_cls.bin', 'wb') as f_out:
            pickle.dump(svc_clf, f_out)

        mlflow.log_artifact('models/svc_cls.bin', artifact_path='best_models')

        mlflow.sklearn.log_model(svc_clf, input_example=X_train_tr[[0]], artifact_path='artifacts')
    
    return {'loss': auc, 'status': STATUS_OK}

In [22]:
svc_result = fmin(
    fn=objective,
    space=svc_params,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  2%|█▌                                                                          | 1/50 [00:01<01:33,  1.91s/trial, best loss: 0.5951511286332715]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000369 seconds

INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.595151



  4%|███                                                                         | 2/50 [00:03<01:31,  1.90s/trial, best loss: 0.5951511286332715]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000377 seconds

INFO:hyperopt.tpe:TPE using 2/2 trials with best loss 0.595151



  6%|████▌                                                                       | 3/50 [00:05<01:30,  1.92s/trial, best loss: 0.5951511286332715]

...

 96%|████████████████████████████████████████████████████████████████████████   | 48/50 [01:32<00:03,  1.92s/trial, best loss: 0.5855725623582766]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001010 seconds

INFO:hyperopt.tpe:TPE using 48/48 trials with best loss 0.585573



 98%|█████████████████████████████████████████████████████████████████████████▌ | 49/50 [01:34<00:01,  1.92s/trial, best loss: 0.5818378942486085]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000415 seconds

INFO:hyperopt.tpe:TPE using 49/49 trials with best loss 0.581838



100%|███████████████████████████████████████████████████████████████████████████| 50/50 [01:36<00:00,  1.92s/trial, best loss: 0.5818378942486085]
